# 🦜🔗 LangChain agent interacting with gmail.

Langchain stellt eine ganze Palette an Toolkits bereit die es Agenten ermöglicht, mit externen Services wie zum Beispiel Gmail, Office365, etc. zu interagieren.
Ein Toolkit ist im Grunde eine Sammlung mehrerer Tools, die man gesammelt einem Agenten zur Verfügung stellt werden können.

In diesem Beispiel wollen wir uns das GmailToolkit ansehen.


In [1]:
# %pip install --upgrade --quiet  google-api-python-client > /dev/null
# %pip install --upgrade --quiet  google-auth-oauthlib > /dev/null
# %pip install --upgrade --quiet  google-auth-httplib2 > /dev/null

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

Zuerst müssen wird das Toolkit konfigurieren, sodass es per Gmail API mit unserem Google Konto kommunizieren kann.

In [3]:
from langchain_community.tools.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)

# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
credentials = get_gmail_credentials(
    token_file="token.json",
    scopes=["https://mail.google.com/"],
    client_secrets_file="credentials.json",
)
api_resource = build_resource_service(credentials=credentials)

Jetzt initialisieren wird das Toolkit und übergeben die vorher konfigurierten API Settings.

Im Output sehen wir, welche Tools über das Toolkit bereit gestellt werden.

In [4]:
from langchain_community.agent_toolkits import GmailToolkit
from langgraph.prebuilt import ToolExecutor

toolkit = GmailToolkit(api_resource=api_resource)
tools = toolkit.get_tools()
tools_executor = ToolExecutor(tools)
tools

[GmailCreateDraft(api_resource=<googleapiclient.discovery.Resource object at 0x10e3d8890>),
 GmailSendMessage(api_resource=<googleapiclient.discovery.Resource object at 0x10e3d8890>),
 GmailSearch(api_resource=<googleapiclient.discovery.Resource object at 0x10e3d8890>),
 GmailGetMessage(api_resource=<googleapiclient.discovery.Resource object at 0x10e3d8890>),
 GmailGetThread(api_resource=<googleapiclient.discovery.Resource object at 0x10e3d8890>)]

Als nächstes erstellen wir unseren Agenten Prompt. Hierzu laden wir einmal ein openai-functions-template aus dem hub.
Zusätzlich legen wir noch instructions fest.

In [5]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from helpers import llm

instructions = """You are an email writer assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

Letzlich erstellen wir einen Langchain Graphen. Hier nutzen wir wieder einen prebuilt Graphen, den wir in diesem Schritt auch ausführen.

In [6]:
from langgraph.prebuilt import create_agent_executor

model = llm(model="gpt-3.5-turbo")
agent_runnable = create_openai_functions_agent(model, tools, prompt)

app = create_agent_executor(agent_runnable, tools)

inputs = {
    "input": """Create an email draft to donald@trump.comm about a contract building a pyramid in Egypt. 
            Please argue why we are the best company for the job. 
            Under no circumstances may you send the message, however. """,
    "chat_history": []
}

app.invoke(inputs)

{'input': 'Create an email draft to donald@trump.comm about a contract building a pyramid in Egypt. \n            Please argue why we are the best company for the job. \n            Under no circumstances may you send the message, however. ',
 'chat_history': [],
 'agent_outcome': AgentFinish(return_values={'output': 'The email draft to Donald Trump regarding the contract to build a pyramid in Egypt has been created. The draft includes arguments on why our company is the best for the job. \n\nPlease review the draft and make any changes if necessary before sending it.'}, log='The email draft to Donald Trump regarding the contract to build a pyramid in Egypt has been created. The draft includes arguments on why our company is the best for the job. \n\nPlease review the draft and make any changes if necessary before sending it.'),
 'intermediate_steps': [(AgentActionMessageLog(tool='create_gmail_draft', tool_input={'message': 'Dear Donald Trump,\n\nI hope this email finds you well. We ar